<a href="https://colab.research.google.com/github/mam-7/CE888/blob/master/CE888/CE888_1907191_ASSIGN/INTERACTIVE_MUSHROOM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd

In [3]:
path = 'gdrive/My Drive/CHECK/'

In [4]:
df=pd.read_csv(path+"mushrooms.csv")

In [5]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
#DATA PREPROCESSING :SINCE THE VALUES ARE STRINGS WE CHANGE TO NUMBERS
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
for col in df.columns:
    df[col] = labelencoder.fit_transform(df[col])
 
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,0,3,2,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,0,2,2,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,0,2,2,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,0,3,2,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,1,3,2,2,7,7,0,2,1,0,3,0,1


In [7]:
X = df.iloc[:,1:23] 
y = df.iloc[:, 0]  
X.head()
y.head()

0    1
1    0
2    0
3    1
4    0
Name: class, dtype: int64

In [8]:
#SCALING FOR BETTER RESULTS IN MODEL
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X=scaler.fit_transform(X)
X

array([[ 1.02971224,  0.14012794, -0.19824983, ..., -0.67019486,
        -0.5143892 ,  2.03002809],
       [ 1.02971224,  0.14012794,  1.76587407, ..., -0.2504706 ,
        -1.31310821, -0.29572966],
       [-2.08704716,  0.14012794,  1.37304929, ..., -0.2504706 ,
        -1.31310821,  0.86714922],
       ...,
       [-0.8403434 ,  0.14012794, -0.19824983, ..., -1.50964337,
        -2.11182722,  0.28570978],
       [-0.21699152,  0.95327039, -0.19824983, ...,  1.42842641,
         0.28432981,  0.28570978],
       [ 1.02971224,  0.14012794, -0.19824983, ...,  0.16925365,
        -2.11182722,  0.28570978]])

In [9]:
#DIMENSION REDUCTION
from sklearn.decomposition import PCA
pca = PCA()
pca.fit_transform(X)

array([[-5.74321902e-01, -9.75781349e-01, -1.22176154e+00, ...,
        -2.08581362e-01,  8.13996758e-03, -1.59211948e-17],
       [-2.28210230e+00,  2.79066333e-01, -1.20049669e+00, ...,
         1.52238789e-01, -1.96446132e-01,  4.35476041e-17],
       [-1.85803562e+00, -2.70972362e-01, -1.37237069e+00, ...,
         2.57581784e-01, -3.62577199e-01,  1.93182410e-17],
       ...,
       [-1.62151632e+00, -7.57536709e-01,  2.73357994e+00, ...,
        -1.42532241e+00,  6.36990122e-01,  7.01453987e-19],
       [ 3.67060561e+00, -1.03277450e+00,  1.68459501e-01, ...,
         9.41440123e-02, -6.43462238e-02, -3.98483680e-20],
       [-1.57520272e+00, -1.22858140e+00,  2.44722789e+00, ...,
        -8.04626064e-01,  5.90315263e-01,  7.43806407e-20]])

In [10]:
!pip install modAL

In [11]:
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling 
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns


In [12]:
from sklearn.model_selection import train_test_split


In [13]:
import numpy as np

In [14]:
#SPLIT THE DATASET, CREATE THE LABELLED AND UNLABELLED POOL FOR QUERYING
n_initial = 100
import itertools
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,shuffle=False)

initial_idx = np.random.choice(range(len(X_train)), size=n_initial, replace=False)

X_initial, y_initial = X_train[initial_idx], y_train[initial_idx]
X_pool = np.array(list(itertools.compress(X, [i not in initial_idx for i in range(len(X_train))])))
y_pool = np.array(list(itertools.compress(y, [i not in initial_idx for i in range(len(y_train))])))


In [15]:
#UNCERTAINTY SAMPLING, CAN BE OUR OWN CHOICE
learner = ActiveLearner(
    estimator=RandomForestClassifier(),
    query_strategy=uncertainty_sampling,
    X_training=X_initial, y_training=y_initial
)

In [16]:
n_queries = 20

In [ ]:
#THIS IS THE INTERACTIVE PART WHERE THE ORACLE WITH SPECIALIZED KNOWLEDGE CAN INPUT THE LABEL THEY THINK IS CORRECT
accuracy_scores = [learner.score(X_test, y_test)]

for i in range(n_queries):
    #display.clear_output(wait=True)
    query_idx, query_inst = learner.query(X_pool)
    print (query_inst)

    print("Is it edible 1 or 0?")
    y_new = np.array([int(input())], dtype=int)
    learner.teach(query_inst.reshape(1, -1), y_new)
    X_pool, y_pool = np.delete(X_pool, query_idx, axis=0), np.delete(y_pool, query_idx, axis=0)
    accuracy_scores.append(learner.score(X_test, y_test))


[[ 1.02971224 -1.48615695 -0.59107461 -0.84322964 -1.49494052  0.16289645
   2.27861212  1.49468272  1.18337533 -1.14480575 -0.10348154  0.68377765
   0.58638466  0.62244139  0.63199138  0.          0.14203663 -0.25613174
   0.94808086 -0.2504706   0.28432981 -0.8771691 ]]
Is it edible 1 or 0?


In [ ]:
#PLOT ACCURACY AS TIME PASSES
with plt.style.context('seaborn-white'):
    plt.figure(figsize=(10, 5))
    plt.title('Accuracy of the classifier during the active learning')
    plt.plot(range(n_queries+1), accuracy_scores)
    plt.scatter(range(n_queries+1), accuracy_scores)
    plt.xlabel('number of queries')
    plt.ylabel('accuracy')
    plt.show()